In [1]:
import pandas as pd 
import numpy as np 
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image
import warnings


# VERIFICA PRESENZA DI CUDA
Con nvidia-smi nel prompt si vede anche la versione di Cuda

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# DOWNLOADING DATASET

In [3]:
train_data= datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor(),)

test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor(),)

100.0%
100.0%
100.0%
100.0%


# DATA LOADER

In [4]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


def handle_dataset():
    train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor(), )
    test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor(), )

    labels_map = { # Classes need to be predicted
        0: 'T-shirt',
        1: 'Trouser',
        2: 'Pullover',
        3: 'Dress',
        4: 'Coat',
        5: 'Sandal',
        6: 'Shirt',
        7: 'Sneaker',
        8: 'Bag',
        9: 'Ankle Boot',
    }
    batch_size = 128  # For processing simultaneously 128 images at every weigth update

    train_dataloader = DataLoader(train_data, batch_size=batch_size,
                                  shuffle=True)  # For every iteration, dataset is divided into gropus of 128 samples. Shuffle helps generalizing the model

    test_dataloader = DataLoader(test_data, batch_size=batch_size)  # Same as train_dataloader but for the test

    return train_dataloader, test_dataloader, labels_map


# DEFYINING THE INITIALIZATION KERNEL AND WEIGHTS
We'll have 1 input channel, since the images are in grayscale, and we select By hand the kernel size, which in this case is in the size of 4 (heights and width are multiple of 4)

For understanding how will be the size of the image after applying a convolution layer, we have to see this formula:
$$O = \frac{(I - K + 2P)}{S} + 1$$

Where *I* is the size of the input, *K* is the size of the kernel, *P* is the padding and *S* is the stride


In [5]:
test_convolutional_layer = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1) # Defines our kernel for our first layer of convolution. 
test_max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

x = next(train_dataloader.__iter__())[0]
print('Shape of x:', x.shape)
## The shape will be 128, 1, 28, 28 because we have 128 images in a batch, 1 channel, and 28x28 pixels

x2 = test_convolutional_layer(x)
print('Shape of x2:', x2.shape)
## The shape will be 128, 3, 26, 26 because we have 128 images in a batch, 5 channels, and 26x26 pixels

x3 = test_max_pool(x2)
print("shape of x3: ", x3.shape)

NameError: name 'train_dataloader' is not defined

# Bulding the Neural Net
Here is were the different net classes (CNNs) are declared. We'll divide the architectures into 2 sets: A1 and A2.
CNNs of A1 share the same architectures, but they differ in the initialization/training of the kernels, and the same happens for A2. 
We have 3 types of the initialization schemas:
-  HF;
- HT;
- DT

NB: WHEN YOU HAVE DECIDED TO USE THE DESIRED ARCHITECTURE, CHANGE THE NAME OF THE CLASS WITH "Net"

## SET A1
### HF

In [ ]:
from torch import nn
import torch

#A1_HF
class Net(nn.Module):
    def __init__(self, classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)  # Convolution layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(5 * 14 * 14, 100)  # first number is to decoding the 3d tensor vector into a 1D dimensional vector, 100 is the number of output neurons of fc1. I chosed 100 because is a good tradeoff between speed and leaning capacity
        self.fc2 = nn.Linear(100, len(classes))  # Final fully connected layer. This is the layer that makes the preictions.
        self.relu = nn.ReLU()  # Activation Function
        self.flatten = nn.Flatten()


    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))

        x = self.flatten(x)  # x becomes a 2D vector

        x = self.relu(self.fc1(x))  # actv. function applied on the first fully connected layer of output

        x = self.fc2(x)  # Prediction
        return x




### HT

In [ ]:
from torch import nn
import torch

#A1_HT
class A1_HT(nn.Module):
    def __init__(self, classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)  # Convolution layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(5 * 14 * 14, 100)  # first number is to decoding the 3d tensor vector into a 1D dimensional vector, 100 is the number of output neurons of fc1. I chosed 100 because is a good tradeoff between speed and leaning capacity
        self.fc2 = nn.Linear(100, len(classes))  # Final fully connected layer. This is the layer that makes the preictions.
        self.relu = nn.ReLU()  # Activation Function
        self.flatten = nn.Flatten()

    def set_initial_kernels(self):
        # Definito by hand i primi 5 kernels
        kernel1 = torch.tensor([[0, 1, 0],
                                [1, 1, 1],
                                [0, 1, 0]], dtype=torch.float32)

        kernel2 = torch.tensor([[1, 0, 1],
                                [0, 1, 0],
                                [1, 0, 1]], dtype=torch.float32)

        kernel3 = torch.tensor([[1, 1, 1],
                                [0, 0, 0],
                                [1, 1, 1]], dtype=torch.float32)

        kernel4 = torch.tensor([[0, 1, 0],
                                [0, 0, 0],
                                [1, 0, 1]], dtype=torch.float32)

        kernel5 = torch.tensor([[1, 1, 0],
                                [1, 0, 0],
                                [0, 0, 0]], dtype=torch.float32)
        kernels = [kernel1, kernel2, kernel3, kernel4, kernel5]  # list of

        with torch.no_grad():
            # I pesi di conv1 hanno shape [5, 1, 3, 3] (5 kernels, 1 canale, dimensione 3x3)
            # Assegno ciascun pattern al corrispondente kernel per l'unico canale in ingresso.
            for k, kernel in enumerate(kernels):
                self.conv1.weight[k, 0] = kernel



    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))

        x = self.flatten(x)  # x becomes a 2D vector

        x = self.relu(self.fc1(x))  # actv. function applied on the first fully connected layer of output

        x = self.fc2(x)  # Prediction
        return x




### DT

In [ ]:
from torch import nn
import torch

#A1_DT
class A1_DT(nn.Module):
    def __init__(self, classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)  # Convolution layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(5 * 14 * 14, 100)  # first number is to decoding the 3d tensor vector into a 1D dimensional vector, 100 is the number of output neurons of fc1. I chosed 100 because is a good tradeoff between speed and leaning capacity
        self.fc2 = nn.Linear(100, len(classes))  # Final fully connected layer. This is the layer that makes the preictions.
        self.relu = nn.ReLU()  # Activation Function
        self.flatten = nn.Flatten()


    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))

        x = self.flatten(x)  # x becomes a 2D vector

        x = self.relu(self.fc1(x))  # actv. function applied on the first fully connected layer of output

        x = self.fc2(x)  # Prediction
        return x




## Set A2
### HF

In [ ]:
from torch import nn
import torch

#A2_HF
class A2_HF(nn.Module):
    def __init__(self, classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)  # Convolution layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(5 * 14 * 14, 100)  # first number is to decoding the 3d tensor vector into a 1D dimensional vector, 100 is the number of output neurons of fc1. I chosed 100 because is a good tradeoff between speed and leaning capacity
        self.fc2 = nn.Linear(100, len(classes))  # Final fully connected layer. This is the layer that makes the preictions.
        self.relu = nn.ReLU()  # Activation Function
        self.flatten = nn.Flatten()

    def set_initial_kernels(self):
        # Definito by hand i primi 5 kernels
        kernel1 = torch.tensor([[0, 1, 0],
                                [1, 1, 1],
                                [0, 1, 0]], dtype=torch.float32)

        kernel2 = torch.tensor([[1, 0, 1],
                                [0, 1, 0],
                                [1, 0, 1]], dtype=torch.float32)

        kernel3 = torch.tensor([[1, 1, 1],
                                [0, 0, 0],
                                [1, 1, 1]], dtype=torch.float32)

        kernel4 = torch.tensor([[0, 1, 0],
                                [0, 0, 0],
                                [1, 0, 1]], dtype=torch.float32)

        kernel5 = torch.tensor([[1, 1, 0],
                                [1, 0, 0],
                                [0, 0, 0]], dtype=torch.float32)
        kernels = [kernel1, kernel2, kernel3, kernel4, kernel5]  # list of

        with torch.no_grad():
            # I pesi di conv1 hanno shape [5, 1, 3, 3] (5 kernels, 1 canale, dimensione 3x3)
            # Assegno ciascun pattern al corrispondente kernel per l'unico canale in ingresso.
            for k, kernel in enumerate(kernels):
                self.conv1.weight[k, 0] = kernel



    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))

        x = self.flatten(x)  # x becomes a 2D vector

        x = self.relu(self.fc1(x))  # actv. function applied on the first fully connected layer of output

        x = self.fc2(x)  # Prediction
        return x




### HT

In [ ]:
from torch import nn
import torch

# A2_HT
class A2_HT(nn.Module):
    def __init__(self, classes):
        super(A2_HT, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)  # Convolution layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(5 * 14 * 14, 100)  # first number is to decoding the 3d tensor vector into a 1D dimensional vector, 100 is the number of output neurons of fc1. I chosed 100 because is a good tradeoff between speed and leaning capacity
        self.fc2 = nn.Linear(100, len(classes))  # Final fully connected layer. This is the layer that makes the preictions.
        self.relu = nn.ReLU()  # Activation Function
        self.flatten = nn.Flatten()

    def set_initial_kernels(self):
        # Definito by hand i primi 5 kernels
        kernel1 = torch.tensor([[0, 1, 0],
                                [1, 1, 1],
                                [0, 1, 0]], dtype=torch.float32)

        kernel2 = torch.tensor([[1, 0, 1],
                                [0, 1, 0],
                                [1, 0, 1]], dtype=torch.float32)

        kernel3 = torch.tensor([[1, 1, 1],
                                [0, 0, 0],
                                [1, 1, 1]], dtype=torch.float32)

        kernel4 = torch.tensor([[0, 1, 0],
                                [0, 0, 0],
                                [1, 0, 1]], dtype=torch.float32)

        kernel5 = torch.tensor([[1, 1, 0],
                                [1, 0, 0],
                                [0, 0, 0]], dtype=torch.float32)

        kernels = [kernel1, kernel2, kernel3, kernel4, kernel5]  # list of

        for k, kernel in enumerate(kernels):
            self.conv1.weight[k, 0] = kernel

    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))

        x = self.flatten(x)  # x becomes a 2D vector

        x = self.relu(self.fc1(x))  # actv. function applied on the first fully connected layer of output

        x = self.fc2(x)  # Prediction
        return x




### DT

In [ ]:
from torch import nn
import torch

#A2_DT
class A2_DT(nn.Module):
    def __init__(self, classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)  # Convolution layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(5 * 14 * 14, 100)  # first number is to decoding the 3d tensor vector into a 1D dimensional vector, 100 is the number of output neurons of fc1. I chosed 100 because is a good tradeoff between speed and leaning capacity
        self.fc2 = nn.Linear(100, len(classes))  # Final fully connected layer. This is the layer that makes the preictions.
        self.relu = nn.ReLU()  # Activation Function
        self.flatten = nn.Flatten()


    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))

        x = self.flatten(x)  # x becomes a 2D vector

        x = self.relu(self.fc1(x))  # actv. function applied on the first fully connected layer of output

        x = self.fc2(x)  # Prediction
        return x




In [ ]:
#net.to(device) # For having an idea about the structure of the net.
# Start_dim = 1 for not flattening the batch size.
# end_dim = -1 for flattening until the last dimension.
# ex: (128, 5, 14, 14) --> (128, 980)

In [ ]:
#print(net.conv1.weight.shape)


# TRAIN_LOOP

In [ ]:
import torch
import os

def train_loop(dataloader, model, loss_fn, optimizer, epoch, device):
    size = len(dataloader.dataset)
    print(f"Training set of size: {size}")

    for batch, (X, y) in enumerate(dataloader):  # (X = input, y = target)
        X, y = X.to(device), y.to(device)  # Setting of 2 architectures

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()  # Loss function calculating the zero-gradient descent
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:  # every 1000 batch it prints the loss
            loss, current = loss.item(), (batch + 1) * len(X)
            current_loss = current / size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    # torch save model with torch.save()
    checkpoint = {
        'epoch': epoch,  # l'epoca corrente
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }

    # Definisci il percorso della cartella dei checkpoint
    checkpoint_dir = r'C:\Users\stefa\Desktop\DNN2025\DNNStefano\DNN\CheckpointsNotebook'

    # Se la cartella non esiste, la creiamo
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Costruiamo il percorso completo del file checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f'epoch_{epoch}_Model_CNN_A1_DT.pt')

    torch.save(checkpoint, checkpoint_path)




# TEST_LOOP

In [ ]:
import torch


def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return 100 * correct, test_loss


# START

In [6]:
from DNNStefano.DNN.SetA1.models.CNN_HF.data_loader import handle_dataset
from DNNStefano.DNN.SetA1.models.CNN_HF.test import test_loop
from DNNStefano.DNN.SetA1.models.CNN_HF.train import train_loop
from DNNStefano.DNN.SetA1.models.CNN_HF.CNN import Net
import torch
from torch import nn
import matplotlib
import matplotlib.pyplot as plt


def start(epochs, iteratore, device, train_loader, test_loader):
    for iterator in range(iteratore, epochs):
        print(f"Epoch {iterator + 1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer, iterator + 1, device)
        accuracy, loss = test_loop(test_dataloader, model, loss_fn, device)
        accuracies.append(accuracy)
        losses.append(loss)
    print("Done!")


if __name__ == "__main__":
    print("START")
    train_dataloader, test_dataloader, labels_map = handle_dataset()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Net(labels_map)  # Dichiarazione di un oggetto di tipo Net
    model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    model.set_initial_kernels()
    learning_rate = 1e-4
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    epochs = 20
    accuracies = []
    losses = []
    iterator = 0
    start(epochs, iterator, device, train_dataloader, test_dataloader)

    epochs = range(1, len(accuracies) + 1)

    # comment if it creates problems
    matplotlib.use('TkAgg')  # Oppure 'Qt5Agg' se hai PyQt5 installato

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)  # Primo subplot
    plt.plot(epochs, accuracies, marker='o', label="Accuracy")
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title(f'Accuracy Over Epochs')
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 2, 2)  # Secondo subplot
    plt.plot(epochs, losses, marker='o', label="Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Loss Over Epochs')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()


START A1_CNN_HF
Epoch 1
-------------------------------
Training set of size: 60000
loss: 2.386296  [  128/60000]
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.628638 

Epoch 2
-------------------------------
Training set of size: 60000
loss: 0.496888  [  128/60000]
Test Error: 
 Accuracy: 80.9%, Avg loss: 0.539108 

Epoch 3
-------------------------------
Training set of size: 60000
loss: 0.433041  [  128/60000]
Test Error: 
 Accuracy: 81.9%, Avg loss: 0.507820 

Epoch 4
-------------------------------
Training set of size: 60000
loss: 0.472780  [  128/60000]
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.485265 

Epoch 5
-------------------------------
Training set of size: 60000
loss: 0.338189  [  128/60000]
Test Error: 
 Accuracy: 83.5%, Avg loss: 0.464642 

Epoch 6
-------------------------------
Training set of size: 60000
loss: 0.402972  [  128/60000]
Test Error: 
 Accuracy: 83.4%, Avg loss: 0.464683 

Epoch 7
-------------------------------
Training set of size: 60000
loss: 0.291120  

# Per ripartire da un checkpoint se per qualche motivo si è bloccato l'addestramento

In [ ]:
"""
# Sostituisci 'epoch_X_Model_CNN_A1_HF.pt' con il nome effettivo del file checkpoint che vuoi ispezionare
checkpoint_path = r'Checkpoints\epoch_3_Model_CNN_A1_HF.pt'
checkpoint = torch.load(checkpoint_path)

# Ripristina lo stato del modello
model.load_state_dict(checkpoint['model_state_dict'])

# Ripristina lo stato dell'ottimizzatore
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Definisce da quale epoca ripartire
restart = checkpoint['epoch']

print(f"Riprendo il training dall' epoca {restart}")

start(epochs, restart)
"""